# P308 LR Vowel2
+ Try to improve F1 for Vowel2

In [1]:
import numpy as np
import pandas as pd
import helper
import submission
import re
import pickle
def split_to_list(curr_row):
    word = curr_row[:curr_row.index(':')]
    splitted_list = curr_row[curr_row.index(':') + 1 : ].split(' ')
    return word, splitted_list

def check(splitted_list):
    count = 0
    target = 0
    removed_stress = []
    for item in splitted_list[:]:
        try: # vowel
            curr = int(item[-1]) 
            removed_stress.append(item[:-1])
            count += 1
            if curr == 1:
                target = count
        except: # consonant
            removed_stress.append(item)
    return removed_stress, target
def count_vowel(removed_stress, vowel):
    count = 0
    for item in removed_stress[:]:
        if item in vowel:
            count += 1
    return count
def singular_filter_sbbb(word, removed_stress, splitted_list):
    
    ## 这里修改下, 若单词长度 <= 3, 不进行消除
    if len(word) <= 3:
        return word, removed_stress
    
    sub_string1 = word[-2: ]
    sub_string2 = word[-3: ]
    sub_string3 = word[-6: ]    
    sub_string4 = word[-4: ]
    
    pattern1 = re.compile('[BCDFGHJKLMNOPQRTVWYZ]{1}S$')
    pattern2 = re.compile('XS$')
    # For CDFGHKLPWRTWYZ --> remove 'S' from word, remove 'S/Z' from removed_stress
    if re.match(pattern1, sub_string1):
        return word[:-1], splitted_list[:-1]
    # For X --> remove 'S' from word, remove 'IH Z' from removed_stress
    if re.match(pattern2, sub_string1):
        return word[:-1], splitted_list[:-2]
    
    # -ES
    pattern3 = re.compile('[ABDEFJKLMNOPQRTUWY]{1}ES$')
    pattern4 = re.compile('IES$')
    pattern5 = re.compile('HES$')
    pattern55 = re.compile('ZZES$|SSES$')
    pattern6 = re.compile('[CGSXZ]{1}ES$')
    pattern7 = re.compile('SELVES$')
    pattern8 = re.compile('IVES$')
    pattern9 = re.compile('VES$')
    
    if re.match(pattern3, sub_string2): # ES --> E
        return word[:-1], splitted_list[:-1]
    if re.match(pattern4, sub_string2): # IES --> Y
        word = word[:-3] + 'Y'
        return word, removed_stress[:-1]
    if re.match(pattern5, sub_string2):
        return word[:-2], removed_stress[:-2]
    if re.match(pattern55, sub_string4):
        return word[:-2], removed_stress[:-2]
    if re.match(pattern6, sub_string2):
        return word[:-1], removed_stress[:-2]
    if re.match(pattern7, sub_string3):
        word = word[:-6] + 'SELF'
        removed_stress.pop()
        removed_stress.pop()
        removed_stress.append('F')
        return word, removed_stress
    if re.match(pattern8, sub_string4) and (removed_stress[-3:] == ['AY', 'V', 'Z']):
        word = word[:-3] + 'FE'
        removed_stress.pop()
        removed_stress.pop()
        removed_stress.append('F')
        return word, removed_stress
    if re.match(pattern9, sub_string2):
        return word[:-1], removed_stress[:-1]
    
    return word, removed_stress
def neutral_filter1(word, removed_stress):
    pattern1 = re.compile('ABL[EY]{1}$')
    if re.search(pattern1, word):
        return word[:-4], removed_stress[:-4]
    
    pattern2 = re.compile('LL[YI]{1}$|LLED$|SSED$|FFED$')
    if re.search(pattern2, word):
        return word[:-2], removed_stress[:-1]
    
    pattern3 = re.compile('L[YI]{1}$')
    if re.search(pattern3, word) and removed_stress[-2:] == ['L', 'IY']:
        return word[:-2], removed_stress[:-2]
    
    pattern4 = re.compile('DDED$|TTED$')
    if re.search(pattern4, word):
        return word[:-3], removed_stress[:-2]
    
    pattern5 = re.compile('[AEIOU]{1}[DFT]{1}ED$')
    if re.search(pattern5, word):
        return word[:-1], removed_stress[:-2]
    
    pattern6 = re.compile('[AEIOU]{1}[KMNS]{1}ED$|[IU]{1}RED$|IBED$')
    if re.search(pattern6, word):
        return word[:-1], removed_stress[:-1]
    
    pattern7 = re.compile('BBED$|GGED$|MMED$|NNED$|PPED$|RRED$')
    if re.search(pattern7, word):
        return word[:-3], removed_stress[:-1]
    
    pattern8 = re.compile('[SW]{1}EED$')
    if re.search(pattern8, word):
        return word[:-4], removed_stress[:-3]
    
    pattern9 = re.compile('B[LR]{1}EED$')
    if re.search(pattern9, word):
        return word[:-5], removed_stress[:-4]
    
    pattern10 = re.compile('[BHKMNOPSWXY]{1}ED$')
    if re.search(pattern10, word):
        return word[:-2], removed_stress[:-1]
    
    pattern11 = re.compile('IED$')
    if re.search(pattern11, word):
        word = word[:-3] + 'Y'
        return word, removed_stress[:-1]
    
    pattern12 = re.compile('[CGLUVZ]{1}ED$')
    if re.search(pattern12, word):
        return word[:-1], removed_stress[:-1]
    
    pattern13 = re.compile('[DFTR]{1}ED$')
    if re.search(pattern13, word):
        return word[:-2], removed_stress[:-2]
    
    pattern14 = re.compile('ISM$')
    if re.search(pattern14, word):
        return word[:-3], removed_stress[:-4]    
    
    pattern15 = re.compile('FUL$|[AO]{1}LF$')
    if re.search(pattern15, word):
        return word[:-3], removed_stress[:-3]  
    
    pattern16 = re.compile('[NL]{1}ESS$') # remove 'NESS', remove 'N AH0 S'
    if re.search(pattern16, word):
        return word[:-4], removed_stress[:-3]
    
    pattern17 = re.compile('ING$')
    if re.search(pattern17, word):
        return word[:-3], removed_stress[:-2]
    
    return word, removed_stress
# 只有singular 和neutral 1
def combined_filter(word, removed_stress, vowel):
    
    if count_vowel(removed_stress, vowel) <= 2:
        return word, removed_stress
    word, removed_stress = singular_filter(word, removed_stress)
    if len(word) == 0 or len(removed_stress) == 0: 
        return 1, [1]
    
    if count_vowel(removed_stress, vowel) <= 2:
        return word, removed_stress
    word, removed_stress = neutral_filter1(word, removed_stress)
    if len(word) == 0 or len(removed_stress) == 0: 
        return 1, [1]
    
    count = 0
    for item in removed_stress[:]:
        if item in vowel:
            count += 1
    if count == 0:
        return 1, [1]
    
    return word, removed_stress

In [2]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score,precision_score, recall_score
vowel = {'AA': 0, 'AW': 1, 'AY': 2, 'ER': 3, 'EY': 4, 'IY': 5, 'OW': 6, 'OY': 7, 'UW': 8, 'AE': 9, 'AH': 10, 'AO': 11, 'EH': 12, 'IH': 13, 'UH': 14}
consonant = {'AA': 0, 'AW': 1, 'AY': 2, 'ER': 3, 'EY': 4, 'IY': 5, 'OW': 6, 'OY': 7, 'UW': 8, 'AE': 9, 'AH': 10, 'AO': 11, 'EH': 12, 'IH': 13, 'UH': 14, 'P': 15, 'B': 16, 'CH': 17, 'D': 18, 'DH': 19, 'F': 20, 'G': 21, 'HH': 22, 'JH': 23, 'K': 24, 'L': 25, 'M': 26, 'N': 27, 'NG': 28, 'R': 29, 'S': 30, 'SH': 31, 'T': 32, 'TH': 33, 'V': 34, 'W': 35, 'Y': 36, 'Z': 37, 'ZH': 38}
raw_data = helper.read_data('asset/training_data.txt')
# split raw data by vowel
def split_raw_by_vowels(raw_data, vowel):
    vowel2 = []
    vowel3 = []
    vowel4 = []
    for curr_row in raw_data[:]:
        word, splitted_list = split_to_list(curr_row)
        removed_stress, target = check(splitted_list)
        num_vowel = count_vowel(removed_stress, vowel)
        if num_vowel == 2:
            vowel2.append(word + ':' + ' '.join(splitted_list))
        if num_vowel == 3:
            vowel3.append(word + ':' + ' '.join(splitted_list))
        if num_vowel == 4:
            vowel4.append(word + ':' + ' '.join(splitted_list))
    return vowel2, vowel3, vowel4
vowel2, vowel3, vowel4 = split_raw_by_vowels(raw_data, vowel)
print(len(vowel2))
print(len(vowel3))
print(len(vowel4))

27619
16395
5986


In [3]:
vowel2[:3]

['COED:K OW1 EH2 D', 'PURVIEW:P ER1 V Y UW2', 'HEHIR:HH EH1 HH IH0 R']

## LR on Vowel2

In [4]:
import helper
import time
import submissionLRvowel2 as submission
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
classifier_path = './asset/classifier.dat'

In [5]:
def split_to_list(curr_row):
    word = curr_row[:curr_row.index(':')]
    splitted_list = curr_row[curr_row.index(':') + 1 : ].split(' ')
    return word, splitted_list
def check(splitted_list):
    count = 0
    target = 0
    removed_stress = []
    for item in splitted_list[:]:
        try: # vowel
            curr = int(item[-1]) 
            removed_stress.append(item[:-1])
            count += 1
            if curr == 1:
                target = count
        except: # consonant
            removed_stress.append(item)
    return removed_stress, target
def get_test_data(test_data):
    data_list = []
    target_list = []
    for curr_row in test_data[:]:
        word, splitted_list = split_to_list(curr_row)
        removed_stress, target = check(splitted_list)
        data_list.append(word + ':' + ' '.join(removed_stress))
        target_list.append(target)
    
    return data_list, target_list  

In [6]:
def fuckfuckrounds(raw_data, epoches):
    for i in range(epoches):
        print("Now round " + str(i))
        print("Start time : " + time.asctime())
        train_data, test_data = train_test_split(raw_data, test_size = 0.1)
        data_list, target_list = get_test_data(test_data)
        submission.train(train_data, classifier_path)
        pred_res = submission.test(data_list, classifier_path)
        acc = accuracy_score(target_list, pred_res)
        f1 = f1_score(target_list, pred_res, average='macro')
        precision = precision_score(target_list, pred_res, average='macro')
        recall = recall_score(target_list, pred_res, average='macro')
        print("Testing accuracy : " + str(acc))
        print("Precision : " + str(precision))
        print("Recall : " + str(recall))
        print("F1 : " + str(f1))
        print("End time : " + time.asctime())
        print("---------------")

In [7]:
classifier_path = './asset/classifier.dat'
fuckfuckrounds(vowel2, epoches = 5)

Now round 0
Start time : Sat May 20 12:11:23 2017


TypeError: 'int' object is not subscriptable